In this notebook, we check if for the census knots the 0-surgery is isotopic to the 0-surgery of its mirror (if the knot is not amphicheiral).

In [1]:
import snappy
import csv
import time

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

### CHECKING FOR ISOMETRIES

def better_is_isometric_to(X,Y,return_isometries=False,index=100,try_hard=False):
    """
    Returns True if X and Y are isometric.
    Returns False if X and Y have different homologies.
     """ 
    if return_isometries==False:
        w='unclear'
        if X.homology()!=Y.homology():
            return False
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w==True:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=1,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=1,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,index=100,try_hard=False)
                    if w==True:
                        return w
        return 'unclear'
    if return_isometries==True:
        w=False
        if X.homology()!=Y.homology():
            return []
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y,return_isometries)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w!=False:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=10,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=10,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,return_isometries,index=100,try_hard=False)
                    if w!=False:
                        return w
        return []
    
def orientation_preserving_isometric(M,N,index=100,try_hard=False):
    '''
    Searches for an orientation preserving isometry.
    '''
    for c in M.dual_curves(max_segments=500):
        P=M.drill(c)
        for d in N.dual_curves(max_segments=500):
            Q=N.drill(d)
            for isom in better_is_isometric_to(P,Q,return_isometries=True,index=index,try_hard=try_hard):
                if isom.extends_to_link():
                    if isom.cusp_maps()[0].determinant()==1:
                        return True
    return False

### SYMMETRY GROUP

def better_symmetry_group(M,index=100):
    '''
    This function computes the symmetry group of the input manifold. 
    If the second entry is True it is proven to be the symmetry group.
    '''
    pos_triang=find_positive_triangulations(M,tries=index)
    if pos_triang==[]:
        full=False
        randomizeCount=0
        while randomizeCount<index and full==False:
            try:
                S=M.symmetry_group()
                full=S.is_full_group()
                M.randomize()
                randomizeCount=randomizeCount+1
            except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
                M.randomize()
                randomizeCount=randomizeCount+1
        try:
            return (S,full)
        except NameError:
            return ('unclear',False)
    X=pos_triang[0]
    full=False
    randomizeCount=0
    while randomizeCount<index and full==False:
        try:
            S=X.symmetry_group()
            full=S.is_full_group()
            X.randomize()
            randomizeCount=randomizeCount+1
        except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
            X.randomize()
            randomizeCount=randomizeCount+1
    if full==True:
        return (S,full)
    pos_triang=find_positive_triangulations(M,number=index,tries=index)
    if pos_triang==[]:
        try:
            return (S,full)
        except NameError:
            return ('unclear',False)
    for X in pos_triang:
        full=False
        randomizeCount=0
        while randomizeCount<index and full==False:
            try:
                S=X.symmetry_group()
                full=S.is_full_group()
                X.randomize()
                randomizeCount=randomizeCount+1
            except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
                X.randomize()
                randomizeCount=randomizeCount+1
        if full==True:
            return (S,full)
    try:
        return (S,full)
    except NameError:
        return ('unclear',False)

In [2]:
signatures=[]

with open('signature.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        signatures.append(row)
signatures=signatures[1:]
len(signatures)

1267

In [3]:
start_time=time.time()
signature_vanishing=[]

for [knot,sig] in signatures:
    if int(sig)==0:
        signature_vanishing.append(knot)
        
print('Number of knots with vanishing signature:',len(signature_vanishing))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60)) 

Number of knots with vanishing signature: 139
Time taken: 6.282329559326172e-06 minutes 


In [4]:
start_time=time.time()
isotopic_to_mirror=[]

for knot in signature_vanishing:
    K=snappy.Manifold(knot)
    try:
        S=K.symmetry_group()
        if S.is_full_group():
            if S.is_amphicheiral():
                isotopic_to_mirror.append(knot)
    except:
        pass
    
print('Number of knots that are isotopic to its mirror:',len(isotopic_to_mirror))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60)) 

Number of knots that are isotopic to its mirror: 5
Time taken: 0.0009117841720581055 minutes 


In [5]:
signature_vanishing_prob_non_amphicheiral=[x for x in signature_vanishing if x not in isotopic_to_mirror]
len(signature_vanishing_prob_non_amphicheiral)

134

We check which of these have hyperbolic and which non-hyperbolic 0-surgeries.

In [6]:
exceptional_fillings=[]
with open('exceptional_fillings.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        exceptional_fillings.append(row)
len(exceptional_fillings)

205823

In [7]:
start_time = time.time()

vanishing_signature_non_hyp=[]
vanishing_signature_hyp=[]

for name in signature_vanishing_prob_non_amphicheiral:
    K=snappy.Manifold(name)
    zero_slope=K.homological_longitude()
    hyperbolic=True
    for x in exceptional_fillings:
        if x[-2]==name:
            if x[-1]==str(zero_slope):
                vanishing_signature_non_hyp.append([name,x[2]])
                hyperbolic=False
            if x[-1]==str((-zero_slope[0],-zero_slope[1])):
                vanishing_signature_non_hyp.append([name,x[2]])
                hyperbolic=False

vanishing_signature_hyp=[x for x in signature_vanishing_prob_non_amphicheiral if x not in [y[0] for y in vanishing_signature_non_hyp]]

print('Number of knots with non hyperbolic 0-surgery:',len(vanishing_signature_non_hyp))
print('Number of knots with hyperbolic 0-surgery:',len(vanishing_signature_hyp))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

Number of knots with non hyperbolic 0-surgery: 56
Number of knots with hyperbolic 0-surgery: 78
Time taken: 0.05833717187245687 minutes 


We first look at the cases where the 0-surgeries are non-hyperbolic. For that we can look at their regina names. 

In [8]:
vanishing_signature_non_hyp

[['m032', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]'],
 ['m074', 'SFS [A: (3,1)] / [ 0,1 | 1,-2 ]'],
 ['m201', 'SFS [S2: (2,1) (5,1) (10,-7)]'],
 ['m222', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ 0,1 | 1,0 ]'],
 ['m372', 'SFS [A: (2,1)] / [ -1,3 | 1,-2 ]'],
 ['s016', 'SFS [A: (4,1)] / [ 0,1 | 1,-2 ]'],
 ['s239', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]'],
 ['s704', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -1,2 | 0,1 ]'],
 ['s879', 'SFS [A: (2,1)] / [ 2,5 | 1,2 ]'],
 ['v0016', 'SFS [A: (5,1)] / [ 0,1 | 1,-2 ]'],
 ['v0321',
  'SFS [D: (2,1) (3,1)] U/m SFS [D: (2,1) (10,7)], m = [ 0,1 | 1,0 ]'],
 ['v1971', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (5,2) (5,3)], m = [ 0,1 | 1,0 ]'],
 ['v2257', 'SFS [A: (5,2)] / [ -1,3 | 1,-2 ]'],
 ['v2272', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm004')])"],
 ['v2362',
  "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,1)]')])"],
 ['v2543',
  'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -2,3 |

If there exists an orientation-reversing diffeomorphism it is isotopic to one that preserves the JSJ decomposition. And in particular there exist orientation reversing-diffeomorphisms of all JSJ pieces. If the piece is a SFS then we can use the classification of SFS with orientation-reversing diffeomorhphisms Section 8 in (https://www.maths.ed.ac.uk/~v1ranick/papers/neumraym.pdf). Thus we can read-off from the regina names directly if a SFS has an orientation-reversing diffeomorphism. By sorting out the SFS that have no orientation-reversing diffeomorphism we are left with the following spaces which all have at least one hyperbolic piece in their JSJ decomposition. We can use SnapPy to check which of these pieces admits an orientation-reversing diffeomorphism.

['v2272', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm004')])"],
    
['t11418', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])"],
 
['o9_22951', "JSJ([('SFSpace', 'SFS [D: (3,1) (3,-1)]'), ('hyperbolic', 'm004')])"],

['o9_27542', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])"],
    
['o9_31828', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])"],

['o9_33568', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])"],
    
['o9_35240', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])"],
 
['o9_36459', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])"],
 
['o9_37768', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])"],

['o9_37795', "JSJ([('hyperbolic', 'm125')])"],
 

In [9]:
hyperbolic_pieces_to_check=[['v2272', 'm004'],
['t11418', 'm043'],
['o9_22951', 'm004'],
['o9_27542', 'm015'],
['o9_31828', 'm032'],
['o9_33568', 'm015'],
['o9_35240', 'm043'],
['o9_36459', 'm043'],
['o9_37768', 'm032'],
['o9_37795', 'm125']]                      

In [10]:
knots_whose_0_surgery_might_admit_reversing_diffeo=[]

for [knot,mfd] in hyperbolic_pieces_to_check:
    M=snappy.Manifold(mfd)
    S=M.symmetry_group()
    if S.is_amphicheiral():
        knots_whose_0_surgery_might_admit_reversing_diffeo.append(knot)
print('Knots whose 0-surgery might admit an orientation-reversing diffeo:',knots_whose_0_surgery_might_admit_reversing_diffeo)

Knots whose 0-surgery might admit an orientation-reversing diffeo: ['v2272', 'o9_22951']


In [11]:
snappy.Manifold('v2272').identify()

[v2272(0,0), K7_85(0,0), K13n469(0,0)]

In [12]:
snappy.Manifold('o9_22951').identify()

[o9_22951(0,0), K9_321(0,0)]

So there are two census knots where we need to check if their 0-surgeries are isotopic to their mirros. The first one is the low-crossing knot we already know. We need to handle the second one.

Next, we look at the hyperbolic cases. Here we check with SnapPy if any of these 0-surgeries admits a diffeomorphism that reverses the orientation.

In [13]:
start_time=time.time()
surgery_isotopic_to_mirror=[]
unclear_symmetry_group=[]

for knot in vanishing_signature_hyp:
    K=snappy.Manifold(knot)
    K.dehn_fill(K.homological_longitude())
    [S,w]=better_symmetry_group(K,index=100)
    if w==True:
        if S.is_amphicheiral():
            print('We found a knot whose 0-surgery IS isotopic to its mirror:',knot)
            surgery_isotopic_to_mirror.append(knot)
    if w!=True:
        print('We could NOT compute the symmetry group of the 0-surgery of:',knot)
        unclear_symmetry_group.append(knot)
    if vanishing_signature_hyp.index(knot)%10==0:
        print('Number of knots checked:',vanishing_signature_hyp.index(knot))
        print('Time taken so far: %s minutes ' % ((time.time() - start_time)/60))  

print('Number of knots that have 0-surgery isotopic to its mirror:',len(surgery_isotopic_to_mirror))
print('Number of knots where we could not compute the symmetry group of the 0-surgery:',len(unclear_symmetry_group))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))  

Number of knots checked: 0
Time taken so far: 0.00011261701583862305 minutes 
We could NOT compute the symmetry group of the 0-surgery of: t00577
Number of knots checked: 10
Time taken so far: 0.9047165195147197 minutes 
We found a knot whose 0-surgery IS isotopic to its mirror: t11462
Number of knots checked: 20
Time taken so far: 0.9053692181905111 minutes 
Number of knots checked: 30
Time taken so far: 0.906231701374054 minutes 
Number of knots checked: 40
Time taken so far: 0.915086583296458 minutes 
Number of knots checked: 50
Time taken so far: 0.9159841497739156 minutes 
Number of knots checked: 60
Time taken so far: 0.9174893339474995 minutes 
We found a knot whose 0-surgery IS isotopic to its mirror: o9_41058
Number of knots checked: 70
Time taken so far: 0.918166704972585 minutes 
Number of knots that have 0-surgery isotopic to its mirror: 2
Number of knots where we could not compute the symmetry group of the 0-surgery: 1
Time taken: 0.9189776698748271 minutes 


For the knot K with unclear symmetry group, we use the following strategy to compute the symmetry group:

We consider the 0-filling K(0,1) and drill out dual curve to get a 1-cusped manifold T.

Then we compute the FPS bound on T and check if the filling that gives back K(0,1) on T is not small with respect to that bound. Then it follows that the symmetry group of T equals the symmetry group of K(0,1).

In [14]:
def better_systole(M,index=100,try_hard=False,cutoff=1.0):
    '''
    Computes the systole of a manifold. 
    It returns False if SnapPy cannot compute the systole.
    '''
    randomizeCount=0
    while randomizeCount<index:
        try:
            systole=M.length_spectrum(cutoff)[0].length.real() 
            return systole
        except IndexError:
            length=2
            while length<10:
                try:
                    systole=better_systole(M,index=100,try_hard=False,cutoff=length) 
                    if systole!=False:
                        return systole
                except (RuntimeError, snappy.SnapPeaFatalError):
                    length=10
                except IndexError:
                    length=length+1
        except (RuntimeError, snappy.SnapPeaFatalError):
            M.randomize()
            randomizeCount=randomizeCount+1
    if try_hard==True:
        pos_triang=find_positive_triangulations(M,number=index,tries=index)
        for X in pos_triang:
            systole=better_systole(X,index)
            if systole!=False:
                return systole
    return False 

def Bound(systole):
    '''
    Takes as input a systole and computes the bound from FPS19.
    '''
    B= math.sqrt(2*math.pi/systole +58)
    if 10.1>B:
        return 10.101
    else:
        return B+0.001

def small_slopes(knot,bound):
    '''
    Returns all hyperbolic slopes of the given knot with length less than the given bound.
    '''
    K=snappy.Manifold(knot)
    return [s for s in K.short_slopes(length=bound*math.sqrt(K.cusp_areas()[0]))[0]]

In [15]:
unclear_symmetry_group

['t00577']

In [16]:
K=snappy.Manifold('t00577')
K.dehn_fill(K.homological_longitude())
print(K,K.homology())

t00577(6,1) Z


In [17]:
P=K.drill(0)
T=P.filled_triangulation()

In [18]:
S=T.symmetry_group()
S

Z/2

In [19]:
S.is_amphicheiral()

False

In [20]:
for s in small_slopes(T,Bound(better_systole(T))):
    T.dehn_fill(s)
    if T.homology()==K.homology():
        print(T,T.homology())

Thus this knot has a 0-surgery that does not admit an orientation-reversing diffeomorphism. We verify that the knots we found are really not isotopic to their mirrors.

In [21]:
surgery_isotopic_to_mirror

['t11462', 'o9_41058']

In [22]:
knots_whose_0_surgery_might_admit_reversing_diffeo

['v2272', 'o9_22951']

In [23]:
for knot in surgery_isotopic_to_mirror+knots_whose_0_surgery_might_admit_reversing_diffeo:
    K=snappy.Manifold(knot)
    D=K.exterior_to_link()
    D.simplify('global')
    print(knot,D)
    print('Jones:',D.jones_polynomial())

t11462 <Link: 1 comp; 20 cross>
Jones: q^-4 - 2*q^-2 + 3 - 2*q^2 + 2*q^4 - q^6 + q^8 - q^10 - q^14 + q^16 - q^18 + q^20
o9_41058 <Link: 1 comp; 26 cross>
Jones: q^-26 - q^-24 + q^-22 - q^-20 - q^-16 + q^-14 - q^-12 + q^-10 + q^-4 - 2*q^-2 + 3 - 2*q^2 + q^4
v2272 <Link: 1 comp; 13 cross>
Jones: q^-14 - q^-12 + q^-10 - q^-8 - q^-2 + 2 - q^2 + q^4
o9_22951 <Link: 1 comp; 31 cross>
Jones: q^-28 - q^-26 + q^-24 - q^-22 + q^-20 - q^-18 - q^-12 + q^-10 - q^-8 + q^-6 + 1 - q^2 + q^4


The Jones polynomials are all not symmetric and thus the knots are not isotopic to their mirrors.

We also remark that these knots have 0 as a symmetry exceptional slope. We confirm this.

In [24]:
K=snappy.Manifold('t11462')
S=K.symmetry_group()
S

Z/2

In [25]:
S.is_amphicheiral()

False

In [26]:
K.dehn_fill(K.homological_longitude())
K

t11462(3,1)

In [27]:
S=K.symmetry_group()
S

D4

In [28]:
S.is_amphicheiral()

True

In [29]:
K=snappy.Manifold('o9_41058')
S=K.symmetry_group()
S

0

In [30]:
K.dehn_fill(K.homological_longitude())
K

o9_41058(3,-1)

In [31]:
S=K.symmetry_group()
S

Z/2

In [32]:
S.is_amphicheiral()

True